In [38]:
import sqlite3 as sql
import pandas as pd
import os

In [43]:
class SQLiteDB:
    
    def __init__(self, db):
        self.db = db
        self.conn = None
        
    def connect(self):
        """Create a database connection to the SQLite database."""
        try:
            self.conn = sql.connect(self.db)
        except sql.Error as e:
            print(e)
            
    def create_table(self, create_table_sql):
        """Create a table from the create_table_sql statement."""
        try:
            c = self.conn.cursor()
            c.execute(create_table_sql)
            print("Table created successfully")
        except sql.Error as e:
            print(e)
    
    def delete_table(self, table_name):
        """Delete a table from the database."""
        try:
            c = self.conn.cursor()
            c.execute(f"DROP TABLE IF EXISTS {table_name}")
            print(f"Table {table_name} deleted successfully")
        except sql.Error as e:
            print(e)
    
    def show_tables(self):
        """Display all tables in the database."""
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        try:
            c = self.conn.cursor()
            c.execute(query)
            tables = c.fetchall()
            print("Tables in the database:")
            for table in tables:
                print(table[0])
        except sql.Error as e:
            print(e)
    
    def close(self):
        """Close the database connection."""
        if self.conn:
            self.conn.close()
            print("Connection Closed")
            
    def insert_csv_data(self, csv_file):
        """
        Add data from a CSV file to a table in the database.
        """
        # Derive table name from the CSV file name (excluding the extension)
        table_name = os.path.splitext(os.path.basename(csv_file))[0]
        
        df = pd.read_csv(csv_file)

        # Insert the data into the table, create the table if it doesn't exist
        df.to_sql(table_name, self.conn, if_exists='append', index=False)

        print(f"Data from {csv_file} added to {table_name} table.")

In [63]:
#create DashMed db

dashmed = SQLiteDB('DashMed.db')

patients = """
CREATE TABLE IF NOT EXISTS patients (
  PatientId integer PRIMARY KEY,
  FirstName text NOT NULL,
  LastName text NOT NULL,
  Address text NOT NULL,
  Phone text NOT NULL,
  Sex text NOT NULL,
  Birthdate date NOT NULL,
  Age integer NOT NULL,
  RelatedPatients text,
  MedicalHistory text,
  Medication text  
);
"""

dashmed.connect()
dashmed.create_table(patients)
dashmed.close()

Table created successfully
Connection Closed


In [64]:
#show tables

dashmed.connect()
dashmed.show_tables()
dashmed.close()

Tables in the database:
patients
Connection Closed


In [65]:
#add data to patients

dashmed.connect()
dashmed.insert_csv_data('patient_data/patients.csv')
dashmed.close()

Data from patient_data/patients.csv added to patients table.
Connection Closed


In [62]:
# confirm db is empty

dashmed.connect()
dashmed.delete_table('patients')
dashmed.close()

Table patients deleted successfully
Connection Closed


In [75]:
class Summary():
    """ Summary class to retrieve patient data from an input patient ID. """
    
    def __init__(self, db):
        self.db = db
        self.PatientId = None

    def input_patient_id(self):
        """ Input the patient ID number. """
        self.PatientId = input('Enter a patient ID: ')

    def patient_exists(self):
        """ Check if the patient ID number does exist in the database. """
        if self.PatientId is None:
            return False
        
        query = 'SELECT COUNT(*) FROM patients WHERE PatientId = ?'  # Placeholder added

        try:
            self.db.connect()
            cursor = self.db.conn.cursor()
            cursor.execute(query, (self.PatientId,))  # Correctly binding the patient ID
            result = cursor.fetchone()
            return result[0] > 0
        
        except sql.Error as e:
            print(e)
            return False

        finally:
            self.db.close()

    def getdata(self):
        """ Return the row containing the patient information as a list. """
        if not self.patient_exists():
            print("Patient ID not valid. Please input valid patient ID.")
            return

        query = 'SELECT * FROM patients WHERE PatientId = ?'  # Placeholder added

        try:
            self.db.connect()
            cursor = self.db.conn.cursor()
            cursor.execute(query, (self.PatientId,))  # Correctly binding the patient ID
            patient_data = [n for n in cursor.fetchone()]
            
            return patient_data
        
        except sql.Error as e:
            print(e)

        finally:
            self.db.close()


In [76]:
test = Summary(dashmed)
test.input_patient_id()

In [77]:
test.getdata()

Connection Closed
Connection Closed


[23989,
 'Edward',
 'Johnson',
 '101 Maple Ave',
 '(555) 123-4567',
 'M',
 '1989-01-24',
 34,
 'Diana Smith',
 'Hypertension',
 'Lisinopril']